This is an application that attempts to simulate the game of k-NIM in the case k = 2. 
feature additions will employ Monte Carlo Methods as we attempt to understand game dynamics further.
Credit to  John Tan Chong Min for his python tutorials and his work in writing a learning program for classical NIM.

In [917]:
#libraries that are used in the program
from copy import deepcopy
import numpy as np
import random


In [918]:
#helpers to avoid duplicates 

#check if two arrays are unordered matches:
def is_match(a,b): 
    match = True
    for item in a: 
        if a.count(item) != b.count(item):
            match = False
            break
    return match

#check if an array has such a match in a collection of other arrays 
def occurs(collection, element):
    has_element = False
    for item in collection:
        if is_match(item, element):
            has_element = True
            break

    return has_element

def all_even(vector):
    result = True
    for x in vector: 
        if x % 2  == 1 :
            result = False
            break
    return result

 

In [919]:
class two_NIM:
    def __init__(self, position):
        self.position = position 
        self.sum = sum(position) 
        self.in_play = True
        self.turn = 1
        self.descendants = self.plays()
        self.reward = 1

    def move(self, play):
        #first we want to assert that the move is valid. 
        # If the program doesn't halt for bad games, our learning program will not function correctly 
        assert(play in self.descendants) 

        self.position = play #update position
        self.sum = sum(play) #update sum 
        self.descendants = self.plays() #update descendants

        if self.over() == True:  #check loss
            self.in_play = False #end game
            if(self.turn == -1):
                self.reward = -1  #punish losers

        self.turn = self.turn * -1  # update turn
        

    def over(self):
        return (self.sum == 0) # game ends when sum accross all heaps is zero

    def reset(self):
        self.sum = sum(self.position)
        self.position = self.turn = 1
        self.in_play = True
        self.reward = 1

    def plays(self): 
        #calculate all descendants: (currently we remove unordered duplicates)
        moves = []
        for index, x in enumerate(self.position):
            child = deepcopy(self.position)
            if(x > 0):
                child[index] -= 1
                if not occurs(moves, child):
                    moves.append(child)

        for indexx, x in enumerate(self.position):
            for indexy, y in enumerate(self.position):
                child = deepcopy(self.position)
                if((x > 0) and (y > 0) and (indexx != indexy)): 
                    child[indexx] -= 1
                    child[indexy] -= 1 
                    if not occurs(moves, child):
                        moves.append(child)
        
        return moves 

In [920]:
#next, we need to make a game method to play the actual games
def Game(P_1, P_2, position, with_script):
    game = two_NIM(position)

    while(game.in_play):
        if (game.turn == 1):
            #print(game.descendants())
            move = P_1(game.descendants)
            game.move(move)
            if(with_script):
                print(f"Player 1 moves to {move}")
        else:
            move = P_2(game.descendants)
            game.move(move)
            if(with_script):
                print(f"Player 2 moves to {move}")
        
    if game.turn == 1 : 
        winner = 2
    else :
        winner = 1

    print(f"Player {winner} wins")

    return game.reward  

In [921]:
# a random player just selects a random position from an array of positions. 
def random_player(position):
    return position[random.randint(0, len(position)-1)]

In [922]:
# now that we ahve a game method, we can play tournaments
def Tournament(P_1, P_2, position, rounds, bool): 

    P_1_score = P_2_score = 0 

    for i in range(0, rounds): 
        result = Game(P_1, P_2, position, bool)
        if(result == 1): 
            P_1_score += 1
            #P_2_score -= 1
        else: 
           # P_1_score -=1
            P_2_score +=1

    print(f"Player 1 wins {P_1_score} games.")
    print(f"Player 2 wins {P_2_score} games.")

    return [f"Player 1 wins {P_1_score} games , Player 2 wins {P_2_score} games"]
    


In [923]:
Tournament(random_player,random_player, [3,3,3], 10, False)

Player 1 wins
Player 2 wins
Player 2 wins
Player 1 wins
Player 1 wins
Player 1 wins
Player 2 wins
Player 2 wins
Player 2 wins
Player 1 wins
Player 1 wins 5 games.
Player 2 wins 5 games.


['Player 1 wins 5 games , Player 2 wins 5 games']

In [924]:
#play a random game and record the reward [1 if win, -1 if loss]
def random_expirament(position):
    if (sum(position) == 0): #just in case we start empty
        return 1
    else: 
        return Game(random_player,random_player, position, True)

In [925]:
def mc_player(position, iterations):
     #iterations = number of times you want to perform the random expirament
     # more iterations means more accurate weights, at the expense of efficiency.
    game = two_NIM(position)
    weights = {} 
    output = []
    for choice in game.descendants: # for each choice (move) in the decisions space (position)
        outcome = 0 
        for x in range(iterations): 
            #print(choice)
            outcome += -(random_expirament(choice)) # play the game randomly
            # it's necessary to factor out the -1 because if a choice is a "good position," then you DON'T want to move there. 
            #print(outcome)

        weights[game.descendants.index(choice)] = outcome/iterations #assign a weight to the choice 
        output.append([choice, weights[game.descendants.index(choice)]])
        
    #print(weights)
    return output

In [926]:
#print(Game(random_player,random_player, [3,3,3], True))
#print(random_expirament([3,3,3]))
#print(mc_player([2,1], 100))
#print(random_player([[1,0],[1,1],[2,0]]))
#print(mc_player([2,5,5], 100))
#print(mc_player([2,1], 10))

In [927]:
class Node:
    def __innit__(self, position, parent, children): 
            # a basic node class each node has a parent and children
            self.parent = parent
            self.position = position 
            self.children = children  #an array

            moves = []
            for index, x in enumerate(self.position):
                child = deepcopy(self.position)
                if(x > 0):
                    child[index] -= 1
                    if not occurs(moves, child):
                        moves.append(child)

            for indexx, x in enumerate(self.position):
                for indexy, y in enumerate(self.position):
                    child = deepcopy(self.position)
                    if((x > 0) and (y > 0) and (indexx != indexy)): 
                        child[indexx] -= 1
                        child[indexy] -= 1 
                        if not occurs(moves, child):
                            moves.append(child)           
            self.children = moves


In [928]:
# MCTS with RAV

